In [1]:
from vertexai.language_models import TextGenerationModel
from vertexai.language_models import TextEmbeddingModel
import vertexai

In [2]:
from langchain.chat_models import ChatVertexAI

In [3]:
from langchain.chat_models import ChatVertexAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)

In [4]:
import sys

# Add the directory to sys.path
sys.path.append('/home/qianyucazelles/chatbot_rag/src')

# Vector store

In [5]:
from utils.chatbot import get_vector_store_by_topic
me = get_vector_store_by_topic(topic="DPO")

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [6]:
q1= '''Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?'''
q2 = '''J'ai des difficultés à comprendre les différentes obligations que j'ai vis-à-vis des sous-traitants qui travaillent pour moi. Pourrais-tu me faire un résumé ?'''
q3 = '''Je ne sais pas quand est-ce qu'une analyse d'impact est indispensable ou non pour un traitement. Peux-tu m'expliquer clairement quand est-ce que je dois les réaliser et de manière simple ?'''

In [7]:
question = q3
context = me.similarity_search(question, k=5)

Waiting


# llm

##### - Chatvertex AI LLM

In [8]:
model_option ="chat-bison" #"gemini-pro" #

In [9]:
llm = ChatVertexAI(model_name=model_option,
                max_output_tokens=1024,
                temperature=0.2,
                top_p=0.75,
                top_k=40,
                verbose=True
            )

/home/qianyucazelles/chatbot_rag/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.vertexai.ChatVertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import ChatVertexAI`.
  warn_deprecated(


#### - local model

In [10]:
# import torch
# from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig, BitsAndBytesConfig
# from transformers import TextStreamer

# torch_dtype = torch.float16
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(device)

# model_id = "HuggingFaceH4/zephyr-7b-beta" #"google/gemma-2b-it"

In [11]:
# device

In [12]:
# tokenizer = AutoTokenizer.from_pretrained(model_id,torch_dtype=torch_dtype)
# model = AutoModelForCausalLM.from_pretrained(model_id, low_cpu_mem_usage=True, torch_dtype=torch_dtype,quantization_config= quantization_config)
# streamer = TextStreamer(tokenizer, skip_prompt=True)

In [13]:
# from langchain.llms import HuggingFacePipeline
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=1024,
#     do_sample= True,
#     temperature=0.2,
#     top_p=0.75,
#     top_k=40,
#     repetition_penalty=1.2,
#     streamer=streamer
# )

# llm = HuggingFacePipeline(pipeline=pipe)

# Conversation Chain

In [14]:
buffer_memory=ConversationBufferWindowMemory(k=3,return_messages=True)

In [15]:
buffer_memory.clear()

In [16]:
def get_template(model_option):
    '''
    Generate prompt template for ConversationChain based on model.
    '''
    human_msg_template = HumanMessagePromptTemplate.from_template(template="{input}")
    if model_option == "chat-bison":
        system_msg_template = SystemMessagePromptTemplate.from_template(
            template="""
            You are an intelligent assistant helping the users with their questions.
            
            Strictly Use ONLY the provided context to answer the question at the end. Think step-by-step and then answer.
        
            Response style:
            - Always citing the document title that you are generating answer from.
            
            Do not try to make up an answer:
            - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
            - If the context is empty, just say "I do not know the answer to that."
            - If it's not a question, just say "Do you want to ask a question?"
            """
        )
        prompt_template = ChatPromptTemplate.from_messages(
            [system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template]
            )
    else:
        prompt_template = ChatPromptTemplate.from_messages(
            [ MessagesPlaceholder(variable_name="history"), human_msg_template]
            )
    return prompt_template

prompt_template = get_template(model_option)
conversation = ConversationChain(memory=buffer_memory, prompt=prompt_template, llm=llm, verbose=True)


In [17]:
question = q1
context = me.similarity_search(question, k=5)

Waiting


In [18]:
context 

[Document(page_content="réception et l'entend dans un délai de deux mois. L'intéressé peut se faire assister par un commissaire aux\ncomptes ou un avocat ou représenter par un avocat.\n \nEn l'absence de motif légitime, la Haute autorité retire le commissaire aux comptes de la liste mentionnée au\nI de l'article L. 821-13.\n \nII.-Lorsqu'un commissaire aux comptes inscrit sur la liste mentionnée au I de l'article L. 821-13 n'a pas\npayé à leur échéance les cotisations dont il est redevable envers la Compagnie nationale ou les compagnies\nrégionales au titre de l'article L. 821-12, le conseil régional met en demeure l'intéressé de respecter ses\nobligations dans un délai de trente jours à compter de la réception de l'acte.\n \nFaute de régularisation dans ce délai, il saisit la Haute autorité de l'audit, qui procède conformément aux\ndeuxième et troisième alinéas du I du présent article.\n \nIII.-Le retrait de la liste mentionnée au I de l'article L", metadata={'source': '../documents/e

In [22]:
context[0].page_content

"réception et l'entend dans un délai de deux mois. L'intéressé peut se faire assister par un commissaire aux\ncomptes ou un avocat ou représenter par un avocat.\n \nEn l'absence de motif légitime, la Haute autorité retire le commissaire aux comptes de la liste mentionnée au\nI de l'article L. 821-13.\n \nII.-Lorsqu'un commissaire aux comptes inscrit sur la liste mentionnée au I de l'article L. 821-13 n'a pas\npayé à leur échéance les cotisations dont il est redevable envers la Compagnie nationale ou les compagnies\nrégionales au titre de l'article L. 821-12, le conseil régional met en demeure l'intéressé de respecter ses\nobligations dans un délai de trente jours à compter de la réception de l'acte.\n \nFaute de régularisation dans ce délai, il saisit la Haute autorité de l'audit, qui procède conformément aux\ndeuxième et troisième alinéas du I du présent article.\n \nIII.-Le retrait de la liste mentionnée au I de l'article L"

In [25]:
context[0].metadata

{'source': '../documents/ecg_dpo/LEGITEXT000005634379.pdf',
 'page': '2693',
 'chunk': '233',
 'title': 'ARTICLE R. 814-58-6 DU CODE DE COMMERCE',
 'score': 0.8135704398155212}

In [21]:
response = conversation.predict(input=f"Context:\n {context} \n\n Query:\n{question}")



> Entering new ConversationChain chain...
Prompt after formatting:
System: 
            You are an intelligent assistant helping the users with their questions.
            
            Strictly Use ONLY the provided context to answer the question at the end. Think step-by-step and then answer.
        
            Response style:
            - Always citing the document title that you are generating answer from.
            
            Do not try to make up an answer:
            - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
            - If the context is empty, just say "I do not know the answer to that."
            - If it's not a question, just say "Do you want to ask a question?"
            
Human: Context:
 [Document(page_content="réception et l'entend dans un délai de deux mois. L'intéressé peut se faire assister par un commissaire aux\ncomptes ou un avocat ou représenter par un avocat.\n \nEn l'ab

In [20]:
print(response)

 Selon l'article R. 814-58-6 du Code de commerce, le délai de deux mois pour répondre à une demande de droit à l'oubli peut être porté à un an, prorogeable de six mois au plus, en cas d'incapacité physique ou légale de la personne titulaire de l'attestation de capacité.
